In [1]:
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('science')

import qiskit as qk

import tools as t

In [2]:
def theoretical(A, B):
    # normalize them
    a = t.normalize(A)
    b = t.normalize(B)
    b_tensor = b.reshape(2, 2)
    
    # reading 1
    wanted = np.einsum('i,ij', a, b_tensor)
    component = wanted / 2
    component_canonical1 = t.canonical(component)
    
    # reading 2
    wanted = np.einsum('i,ji', a, b_tensor)
    component = wanted / 2
    component_canonical2 = t.canonical(component)
    
    return component_canonical1, component_canonical2

def experimental(A, B, ibm=False):
    a = t.normalize(A)
    b = t.normalize(B)
    
    qc = qk.QuantumCircuit(4)
    qc.h(0)
    qc.initialize(a, 1)
    qc.initialize(b, [3, 2])

    qc.cswap(0, 2, 3)

    qc.cx(1, 2)
    qc.h(1)
    
    component_canonical1, component_canonical2 = t.measure_substates(qc, 3, ['000', '100'], shots=2**13, ibm=ibm)
    
    return component_canonical1, component_canonical2

def make_plot(N_points, ibm=False):
    # initialize random tensors and arrays to store results
    A_arr = (np.random.random([N_points, 2])*2 - 1) + (np.random.random([N_points, 2])*2 - 1) * 1j
    B_arr = (np.random.random([N_points, 4])*2 - 1) + (np.random.random([N_points, 4])*2 - 1) * 1j
    reading1_arr_the = np.empty([N_points, 2], dtype=complex)
    reading2_arr_the = np.empty([N_points, 2], dtype=complex)
    reading1_arr_exp = np.empty([N_points, 2], dtype=complex)
    reading2_arr_exp = np.empty([N_points, 2], dtype=complex)
    
    # calculate results
    for i in range(N_points):
        reading1_arr_the[i], reading2_arr_the[i] = theoretical(A_arr[i], B_arr[i])
        reading1_arr_exp[i], reading2_arr_exp[i] = experimental(A_arr[i], B_arr[i], ibm=ibm)
        
    # separate into real and imaginary parts
    real_ket0_the1 = np.real(reading1_arr_the[:,0])
    real_ket1_the1 = np.real(reading1_arr_the[:,1])
    imag_ket1_the1 = np.imag(reading1_arr_the[:,1])

    real_ket0_exp1 = np.real(reading1_arr_exp[:,0])
    real_ket1_exp1 = np.real(reading1_arr_exp[:,1])
    imag_ket1_exp1 = np.imag(reading1_arr_exp[:,1])

    real_ket0_the2 = np.real(reading2_arr_the[:,0])
    real_ket1_the2 = np.real(reading2_arr_the[:,1])
    imag_ket1_the2 = np.imag(reading2_arr_the[:,1])

    real_ket0_exp2 = np.real(reading2_arr_exp[:,0])
    real_ket1_exp2 = np.real(reading2_arr_exp[:,1])
    imag_ket1_exp2 = np.imag(reading2_arr_exp[:,1])
    
    # make beautiful plot
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize = (10, 5), constrained_layout=True, dpi=1200)

    axs[0].set_title('Reading I')
    axs[0].set_xlabel('theoretical')
    axs[0].set_ylabel('quantum')

    axs[0].plot(np.linspace(-0.5, 0.5, 2), np.linspace(-0.5, 0.5, 2), 'k--', linewidth=0.5)
    axs[0].plot(real_ket0_the1, real_ket0_exp1, '.', label=r'real $\left\vert 0\right\rangle $')
    axs[0].plot(real_ket1_the1, real_ket1_exp1, '.', label=r'real $\left\vert 1\right\rangle $')
    axs[0].plot(imag_ket1_the1, imag_ket1_exp1, '.', label=r'img $\left\vert 1\right\rangle $')

    axs[1].set_title('Reading II')
    axs[1].set_xlabel('theoretical')
    axs[1].set_ylabel('quantum')

    axs[1].plot(np.linspace(-0.5, 0.5, 2), np.linspace(-0.5, 0.5, 2), 'k--', linewidth=0.5)
    axs[1].plot(real_ket0_the2, real_ket0_exp2, '.', label=r'real $\left\vert 0\right\rangle $')
    axs[1].plot(real_ket1_the2, real_ket1_exp2, '.', label=r'real $\left\vert 1\right\rangle $')
    axs[1].plot(imag_ket1_the2, imag_ket1_exp2, '.', label=r'img $\left\vert 1\right\rangle $')

    axs[0].legend()
    axs[1].legend()
    
    return fig

In [3]:
fig = make_plot(10)
plt.savefig('notes/figures/simulator.png', format='png', dpi=1200)
plt.plot()

/home/hugens/shared/uni/project/quantum-nlp/tools/clsresults.py:204: RuntimeWarning: invalid value encountered in arccos
  phi_norm = np.absolute(np.arccos(min(1, (p2[0]-p2[1]) / np.sin(theta))))


[]

In [ ]:
fig = make_plot(10, ibm=True)
plt.savefig('notes/figures/ibm.png', format='png', dpi=1200)
plt.plot()